## LSTM with Variable-Length Input to One-Char Output

### Motivation
In last example, 'Stateful LSTM for a One-Char to One-Char Mapping', we observed that when we predict the next letter from "K", it predicts "B" and falls back into reguraitating the entire alphabet. To truly predict 'K' the state of the network would need to be warmed up iteratively fed the letters from "A" to "J". This tells us that we could achieve the same effect with a "stateless" LSTM by preparing training data like 

    ---a -> b
    --ab -> c
    -abc -> d
    abcd -> e
    
Where the input sequence is fixed at 25 (a-to-y to predict z) and patterns are prefixed with zero-padding. Finally this raises the question of training an LSTM network using variable length input sequence to predict the next character. 

### Overview

In this section we explore a variation of the "stateless" LSTM that learns random subsequences of the alphabet and an effort to build a model that can be given arbitrary letters or subsequences of letters and predict the next letter in the alphabet. 

#### Framing the Problem
Firstly, we are changing the framing of the problem. To simplify we will define a maximum input sequence length and set it to a small value like 5 to speed up training. This defines the maximum length of subsequences of the alphabet will be drawn for training. In extensions, this could just as set to the full aphabet(26) or longer if we allow looping back to the start of the sequence. 

We also need to define the number of random sequences to create, in this case 1000. This too could be more or less. I expect less patterns are actually required. 


    # prepare the dataset of input to output pairs encoded as integers
    num_inputs = 1000
    max_len = 5
    dataX = []
    dataY = []
    for i in range(num_inputs):
        start = numpy.random.randint(len(alphabet)-2)
        end = numpy.random.randint(start, min(start+max_len,len(alphabet)-1))
        sequence_in = alphabet[start:end+1]
        sequence_out = alphabet[end + 1]
        dataX.append([char_to_int[char] for char in sequence_in])
        dataY.append(char_to_int[sequence_out])
        print sequence_in, '->', sequence_out



Running this code in the broader context will create input patterns that look like the following:


    PQRST -> U
    W -> X
    O -> P
    OPQ -> R
    IJKLM -> N
    QRSTU -> V
    ABCD -> E
    X -> Y
    GHIJ -> K

The input sequence vary in length between 1 and max_len and there fore require zero padding. here we use left-hand-side (prefix) padding with the Keras built in **pad_sequences()** function. 

    X = pad_sequences(dataX, maxlen=max_len, dtype='float32')
    
 
The trained model is evaluated on randomly selected input patterns. **This could just as easily be new randomly generated sequences of characters.** I also believe this could also be a linear sequence seeded with “A” with outputs fes back in as single character inputs.    


In [7]:
# LSTM with Variable Length Input Sequences to One Character Output
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from theano.tensor.shared_randomstreams import RandomStreams
# fix random seed for reproducibility
numpy.random.seed(7)
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# prepare the dataset of input to output pairs encoded as integers
num_inputs = 1000
max_len = 5
dataX = []
dataY = []
for i in range(num_inputs):
	start = numpy.random.randint(len(alphabet)-2)
	end = numpy.random.randint(start, min(start+max_len,len(alphabet)-1))
	sequence_in = alphabet[start:end+1]
	sequence_out = alphabet[end + 1]
	dataX.append([char_to_int[char] for char in sequence_in])
	dataY.append(char_to_int[sequence_out])
	print(sequence_in, '->', sequence_out)

PQRST -> U
W -> X
O -> P
OPQ -> R
IJKLM -> N
QRSTU -> V
ABCD -> E
X -> Y
GHIJ -> K
M -> N
XY -> Z
QRST -> U
ABC -> D
JKLMN -> O
OP -> Q
XY -> Z
D -> E
T -> U
B -> C
QRSTU -> V
HIJ -> K
JKLM -> N
ABCDE -> F
X -> Y
V -> W
DE -> F
DEFG -> H
BCDE -> F
EFGH -> I
BCDE -> F
FG -> H
RST -> U
TUV -> W
STUV -> W
LMN -> O
P -> Q
MNOP -> Q
JK -> L
MNOP -> Q
OPQRS -> T
UVWXY -> Z
PQRS -> T
D -> E
EFGH -> I
IJK -> L
WX -> Y
STUV -> W
MNOPQ -> R
P -> Q
WXY -> Z
VWX -> Y
V -> W
HI -> J
KLMNO -> P
UV -> W
JKL -> M
ABCDE -> F
WXY -> Z
M -> N
CDEF -> G
KLMNO -> P
RST -> U
RS -> T
W -> X
J -> K
WX -> Y
JKLMN -> O
MN -> O
L -> M
BCDE -> F
TU -> V
MNOPQ -> R
NOPQR -> S
HIJ -> K
JKLM -> N
STUVW -> X
QRST -> U
N -> O
VWXY -> Z
B -> C
UVWX -> Y
OP -> Q
K -> L
C -> D
X -> Y
ST -> U
JKLM -> N
B -> C
QR -> S
RS -> T
VWXY -> Z
S -> T
NOP -> Q
KLMNO -> P
IJ -> K
EF -> G
MNOP -> Q
WXY -> Z
HI -> J
P -> Q
STUVW -> X
Q -> R
MN -> O
O -> P
C -> D
L -> M
JKLM -> N
K -> L
IJKLM -> N
FGHIJ -> K
LM -> N
OPQ -> R
U -> V
HIJ

In [8]:
# convert list of lists to array and pad sequences if needed
X = pad_sequences(dataX, maxlen=max_len, dtype='float32')
# reshape X to be [samples, time steps, features]
X = numpy.reshape(X, (X.shape[0], max_len, 1))
print(X.shape)
for i in range(5):
    print(X[i])

(1000, 5, 1)
[[ 15.]
 [ 16.]
 [ 17.]
 [ 18.]
 [ 19.]]
[[  0.]
 [  0.]
 [  0.]
 [  0.]
 [ 22.]]
[[  0.]
 [  0.]
 [  0.]
 [  0.]
 [ 14.]]
[[  0.]
 [  0.]
 [ 14.]
 [ 15.]
 [ 16.]]
[[  8.]
 [  9.]
 [ 10.]
 [ 11.]
 [ 12.]]


In [9]:
# normalize
X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# create and fit the model
batch_size = 1
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], 1)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, batch_size=batch_size, verbose=2)

Epoch 1/500
5s - loss: 3.0782 - acc: 0.0640
Epoch 2/500
4s - loss: 2.7659 - acc: 0.1300
Epoch 3/500
4s - loss: 2.4372 - acc: 0.1960
Epoch 4/500
4s - loss: 2.2115 - acc: 0.2620
Epoch 5/500
4s - loss: 2.0617 - acc: 0.3090
Epoch 6/500
4s - loss: 1.9389 - acc: 0.3260
Epoch 7/500
4s - loss: 1.8385 - acc: 0.3440
Epoch 8/500
4s - loss: 1.7543 - acc: 0.3770
Epoch 9/500
4s - loss: 1.6747 - acc: 0.4220
Epoch 10/500
4s - loss: 1.5998 - acc: 0.4490
Epoch 11/500
4s - loss: 1.5303 - acc: 0.4730
Epoch 12/500
4s - loss: 1.4696 - acc: 0.4940
Epoch 13/500
4s - loss: 1.4163 - acc: 0.5080
Epoch 14/500
4s - loss: 1.3601 - acc: 0.5460
Epoch 15/500
4s - loss: 1.3137 - acc: 0.5700
Epoch 16/500
4s - loss: 1.2663 - acc: 0.5860
Epoch 17/500
4s - loss: 1.2242 - acc: 0.5830
Epoch 18/500
4s - loss: 1.1875 - acc: 0.6230
Epoch 19/500
4s - loss: 1.1409 - acc: 0.6260
Epoch 20/500
4s - loss: 1.1164 - acc: 0.6460
Epoch 21/500
4s - loss: 1.0755 - acc: 0.6620
Epoch 22/500
4s - loss: 1.0508 - acc: 0.6710
Epoch 23/500
4s - l

In [10]:
# summarize performance of the model
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))

Model Accuracy: 98.50%


In [11]:
# demonstrate some model predictions
for i in range(20):
	pattern_index = numpy.random.randint(len(dataX))
	pattern = dataX[pattern_index]
	x = pad_sequences([pattern], maxlen=max_len, dtype='float32')
	x = numpy.reshape(x, (1, max_len, 1))
	x = x / float(len(alphabet))
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	print(seq_in, "->", result)

['T', 'U', 'V', 'W', 'X'] -> Y
['V', 'W', 'X', 'Y'] -> Z
['A', 'B', 'C', 'D'] -> E
['C'] -> D
['K', 'L', 'M', 'N'] -> O
['B'] -> C
['C', 'D', 'E', 'F', 'G'] -> H
['Q', 'R'] -> S
['T', 'U', 'V', 'W', 'X'] -> Y
['D', 'E', 'F', 'G', 'H'] -> I
['B', 'C', 'D', 'E', 'F'] -> G
['C', 'D', 'E', 'F'] -> G
['C'] -> D
['K', 'L', 'M'] -> N
['B', 'C', 'D', 'E'] -> F
['N', 'O'] -> P
['P'] -> Q
['W'] -> X
['V', 'W', 'X'] -> Y
['C'] -> D


We can see that although the model did not learn the alphabet perfectly from the randomly generated subsequences, it did very well. The model was not tuned and may require more training or a larger network, or both.

This is a good natural extension to the “all sequential input examples in each batch” alphabet model learned above in that it can handle ad hoc queries, but this time of arbitrary sequence length (up to the max length).